# Building Network services



# Socket programmming

If you want to use a protocol that Python doesn't support natively (or just want to use your own protocol), you can always use the lower-level `socket` module in the standard library.

But first, a (brief) review of network protocol layers

![Image](data/img/OSI.png "OSI Stack")

## Basic socket programming

In [1]:
import socket

In [2]:
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock = socket.socket()

In [3]:
sock.connect(('www.cnn.com', 80))
http_req = '''GET / HTTP/1.1
Host: www.cnn.com
User-Agent: Advanced-Python/1.0
Accept: */*

'''

In [4]:
sock.sendall(http_req.encode('utf-8'))
response = sock.recv(1024)
sock.close()
print(len(response))

560


In [5]:
print(response.decode('utf8'))

HTTP/1.1 301 Moved Permanently
Server: Varnish
Retry-After: 0
Content-Length: 0
Cache-Control: public, max-age=600
Location: https://www.cnn.com/
Accept-Ranges: bytes
Date: Thu, 24 Sep 2020 22:29:23 GMT
Via: 1.1 varnish
Connection: close
Set-Cookie: countryCode=US; Domain=.cnn.com; Path=/; SameSite=Lax
Set-Cookie: stateCode=GA; Domain=.cnn.com; Path=/; SameSite=Lax
Set-Cookie: geoData=marietta|GA|30068|US|NA|-400|broadband|33.970|-84.430; Domain=.cnn.com; Path=/; SameSite=Lax
X-Served-By: cache-pdk17868-PDK
X-Cache: HIT
X-Cache-Hits: 0




### Socket programming basics:

#### Client

 - `connect()`
 - `send()`, `recv()`
 - generally does _not_ `bind()` (but may)
 
#### Server

 - `bind()` to a well-known port
 - `listen()` to set up a *connection backlog*
 - `accept()` incoming connections, returning **a new socket**

In [6]:
sock.listen?

In [7]:
from contextlib import closing

def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(0)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
        with closing(peer_sock):
            buffer = peer_sock.recv(1000)
            print('Received "{}"'.format(buffer))
            peer_sock.sendall(buffer)

In [9]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 49830)
Received "b'hello\n'"


### Better: use a handler in a thread

In [10]:
import threading

def handle_echo(sock, addr):
    while True:
        buffer = sock.recv(1000)
        print('Received {}'.format(buffer))
        if not buffer:
            print('Socket closed, exiting thread')
            break
        sock.sendall(buffer)
        
def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(0)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
        t = threading.Thread(target=handle_echo, args=(peer_sock, peer_addr))
        t.start()

In [11]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 49843)
Received b'Hello\n'
Received b'there\n'
Received b'this\n'
Received b'is working\n'
Received b'still here\n'
Received b''
Socket closed, exiting thread


## Using SocketServer as a socket server framework

In [12]:
import socketserver

class MyEchoHandler(socketserver.BaseRequestHandler):
    def handle(self):
        while True:
            buffer = self.request.recv(1000)
            print('Received {}'.format(buffer))
            if not buffer:
                print('Socket disconnected, exiting handler')
                break
            self.request.sendall(buffer)

In [13]:
server = socketserver.TCPServer(('localhost', 8042), MyEchoHandler)

In [14]:
server.serve_forever()

Received b'Hello there\n'
Received b'cool!\n'
Received b''
Socket disconnected, exiting handler
Received b'Multiple connections\n'
Received b''
Socket disconnected, exiting handler


KeyboardInterrupt: 

In [15]:
server = socketserver.ThreadingTCPServer(('localhost', 8043), MyEchoHandler)

In [16]:
server.serve_forever()

Received b'Another one\n'
Received b'Actually multiple!\n'
Received b''
Socket disconnected, exiting handler
Received b''
Socket disconnected, exiting handler


KeyboardInterrupt: 

# UDP  - DNS example

from https://routley.io/posts/hand-writing-dns-messages/

In [17]:
sock = socket.socket(type=socket.SOCK_DGRAM)

In [18]:
msg = '''
AA AA 01 00 00 01 00 00 00 00 00 00
07 65 78 61 6d 70 6c 65 03 63 6f 6d 
00 00 01 00 01
'''

In [19]:
msg = ''.join(msg.split()).encode('utf-8')
msg

b'AAAA01000001000000000000076578616d706c6503636f6d0000010001'

In [22]:
import binascii
b_msg = binascii.unhexlify(msg)
b_msg

b'\xaa\xaa\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x07example\x03com\x00\x00\x01\x00\x01'

In [23]:
sock.sendto(b_msg, ('8.8.8.8', 53))
sock.recvfrom(4096)

(b'\xaa\xaa\x81\x80\x00\x01\x00\x01\x00\x00\x00\x00\x07example\x03com\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00Q\xc1\x00\x04]\xb8\xd8"',
 ('8.8.8.8', 53))

If you _are_ building your own protocol layer, you'll probably want to become familiar with the `struct` module:

```python
import struct

struct.pack(...)
```

In [24]:
import struct

In [25]:
struct.pack('iii', 0x7aaa5555, 0x7aaa5555, 0x7aaa5555)

b'UU\xaazUU\xaazUU\xaaz'

# Lab 

Open the [socket lab][socket-lab]

[socket-lab]: ./socket-lab.ipynb